In [1]:
import os
import time
import librosa
import numpy as np
import tensorflow as tf
import cPickle as pickle
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

## LSTM networks

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [3]:
n_chunks = 28
chunks_size = 28
img_size_flat = n_chunks * chunks_size
n_labels = 10
batch_size = 64
hidden_size = 64

In [4]:
## Place holder variables
X = tf.placeholder(tf.float32, [None, img_size_flat])
X_rnn = tf.reshape(X, [-1, n_chunks, chunks_size])
Y = tf.placeholder(tf.float32, [None, n_labels])

In [5]:
graph = tf.Graph()
with graph.as_default():
    ## Input gate: input, previous output and bias
    ix = tf.Variable(tf.truncated_normal([chunks_size, hidden_size], stddev=0.1))
    im = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1))
    ib = tf.Variable(tf.zeros([1, hidden_size]))

    ## New memory gate: input, previous output and bias
    cx = tf.Variable(tf.truncated_normal([chunks_size, hidden_size], stddev=0.1))
    cm = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1))
    cb = tf.Variable(tf.zeros([1, hidden_size]))

    ## Forget gate: input, previous output and bias
    fx = tf.Variable(tf.truncated_normal([chunks_size, hidden_size], stddev=0.1))
    fm = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1))
    fb = tf.Variable(tf.zeros([1, hidden_size]))

    ## Output gate: input, previous output and bias
    ox = tf.Variable(tf.truncated_normal([chunks_size, hidden_size], stddev=0.1))
    om = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1))
    ob = tf.Variable(tf.zeros([1, hidden_size]))

    ## Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, hidden_size]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, hidden_size]), trainable=False)

    ## Classifier params
    w = tf.Variable(tf.truncated_normal([hidden_size, n_labels], stddev=0.1))
    b = tf.Variable(tf.zeros([n_labels]))
    
    def lstm_cell(i, o, previous_state):
        input_gate = tf.nn.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        newmem_gate = tf.nn.tanh(tf.matmul(i, cx) + tf.matmul(o, cm) + cb)
        forget_gate = tf.nn.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        output_gate = tf.nn.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        new_state = forget_gate*previous_state + input_gate*newmem_gate
        output = output_gate*tf.nn.tanh(new_state)
        return output, state
    
    ## Training data
    train_inputs = list()
    train_labels = list()
    for _ in range(n_chunks):
        train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size, chunks_size]))
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, n_labels]))
        

    ## Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for train_input in train_inputs:
        output, state = lstm_cell(train_input, output, state)
        outputs.append(output)

    ## Save state
    with tf.control_dependencies([saved_output.assign(output),
                                    saved_state.assign(state)]):
     
        logits = tf.matmul(tf.concat(0, outputs), w) + b
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.concat(0,train_labels))
        cost = tf.reduce_mean(cross_entropy)

    #Optimizer

    global_step =tf.Variable(0)
    learning_rate =tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(cost))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    train_prediction = tf.nn.softmax(logits)
    

In [12]:
n_iterations = 5
cost_history = np.empty(shape=[1],dtype=float)

with tf.Session(graph=graph) as session:
    init = tf.global_variables_initializer()
    session.run(init)
    mean_loss = 0
    for itr in range(n_iterations):
        x_batch, y_batch = data.train.next_batch(batch_size)
        x_batch = np.reshape(x_batch, [-1, n_chunks, chunks_size])
        print (np.shape(x_batch[:,:,2]))
        feed_dict_train = dict()
        for i in range(n_chunks):
            feed_dict_train[train_inputs[i]] = x_batch[:,:,i]
            feed_dict_train[train_labels[i]] = y_batch
            _, c, prediction = session.run([optimizer, cost, train_prediction], feed_dict=feed_dict_train)
        cost_history = np.append(cost_history,c)
        

(64, 28)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [64,28]
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[64,28], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_2', defined at:
  File "/home/kyle/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kyle/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kyle/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/kyle/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/kyle/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/kyle/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.